Using the images marked as valid per cluster, we pass them through the CNN and extract their feature vectors. the results are stored at a per-country basis. For example, all Malawi feature extractions will go into results/malawi_2016/cnn.

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
BASE_DIR = '..'
COUNTRY = 'malawi_2016'
METRIC = 'HDDS'
RANDOM_SEED = 7

CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'data', 'cnn_images', COUNTRY, METRIC)
CNN_SAVE_DIR = os.path.join(BASE_DIR, 'models', COUNTRY, METRIC)

COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')
PROCESSED_DIR = os.path.join(COUNTRIES_DIR, COUNTRY, 'processed')
RESULTS_DIR = os.path.join(BASE_DIR, 'results', COUNTRY, METRIC)

In [3]:
os.makedirs(RESULTS_DIR, exist_ok=True)

# Feature extract with CNN
If you have run this step before, you can skip it and run the commented out code in the next section to quick-start.

In [4]:
def load_country(metric):
    filepath = os.path.join(PROCESSED_DIR, f'{metric}.csv')
    df_images = pd.read_csv(filepath)
    return df_images

In [5]:
df_images = load_country(METRIC)

In [6]:
df_images.head()

,country,image_name,image_lat,image_lon,cluster_lat,cluster_lon,rCSI,FCS,HDDS,is_train,bin,near_lower,near_upper
0,malawi_2016,-17.140065764205975_35.17229723579403_-17.0951...,-17.140066,35.172297,-17.09515,35.217213,0.0,32.25,3.75,True,0,False,False
1,malawi_2016,-17.117607882102988_35.17229723579403_-17.0951...,-17.117608,35.172297,-17.09515,35.217213,0.0,32.25,3.75,True,0,False,False
2,malawi_2016,-17.09515_35.17229723579403_-17.09515_35.21721...,-17.095150,35.172297,-17.09515,35.217213,0.0,32.25,3.75,True,0,False,False
3,malawi_2016,-17.072692117897013_35.17229723579403_-17.0951...,-17.072692,35.172297,-17.09515,35.217213,0.0,32.25,3.75,True,0,False,False
4,malawi_2016,-17.050234235794026_35.17229723579403_-17.0951...,-17.050234,35.172297,-17.09515,35.217213,0.0,32.25,3.75,True,0,False,False


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} as backend')
model = torch.load(os.path.join(CNN_SAVE_DIR, f'trained_model_{METRIC}.pt'), map_location=device)

Using cuda as backend


In [8]:
# remove the final layers
model.classifier = model.classifier[:4]

In [9]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
)

In [10]:
input_size = 224
transformer = transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

# custom dataset for fast image loading and processing
# does not follow the usual style of folder -> folder for each class -> image
# we just want one folder with images
class ForwardPassDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, transformer):
        self.image_dir = image_dir
        self.image_list = os.listdir(self.image_dir)
        self.transformer = transformer

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        image_name = self.image_list[index]

        # Load image
        X = self.filename_to_im_tensor(self.image_dir + '/' + image_name)
        
        # dataloaders need to return a label, but for the forward pass we don't really care
        return X, -1
    
    def filename_to_im_tensor(self, file):
        im = (plt.imread(file)[:,:,:3] * 256).astype(np.uint8)
        im = Image.fromarray(im)
        im = self.transformer(im)
        return im
    
def run_forward_pass(model, df_images, mode):
    assert mode in ['train', 'valid']
    model.eval()  
    # shape of final array will be (num_images, 4096)
    # we also want to record the image each index represents
    feats = None
    if mode == 'train':
        feats = np.zeros(((df_images['is_train']).sum(), 4096))
    else:
        feats = np.zeros(((~df_images['is_train']).sum(), 4096))
    image_order = []
    i = 0
    # use the validation images to do the forward pass
    dataset = ForwardPassDataset(os.path.join(CNN_TRAIN_IMAGE_DIR, mode), transformer)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False, num_workers=4)
    image_order += dataset.image_list
    # forward pass for this class
    for inputs, _ in tqdm(dataloader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        feats[i:i+len(inputs),:] = outputs.cpu().detach().numpy()
        i += len(inputs)
            
    forward_pass_df = pd.DataFrame.from_dict({'image_name': image_order, 'feat_index': np.arange(len(image_order))})
    return feats, forward_pass_df

In [11]:
feats_train, df_train = run_forward_pass(model, df_images, 'train')

In [12]:
feats_valid, df_valid = run_forward_pass(model, df_images, 'valid')

In [13]:
df_train = pd.merge(left=df_images, right=df_train, on='image_name', how='inner')

In [14]:
df_valid = pd.merge(left=df_images, right=df_valid, on='image_name', how='inner')

In [15]:
# have we maintained all images?
assert len(df_train) == (df_images['is_train']).sum()
assert len(df_valid) == (~df_images['is_train']).sum()

In [16]:
df_train

,country,image_name,image_lat,image_lon,cluster_lat,cluster_lon,rCSI,FCS,HDDS,is_train,bin,near_lower,near_upper,feat_index
0,malawi_2016,-17.140065764205975_35.17229723579403_-17.0951...,-17.140066,35.172297,-17.095150,35.217213,0.0,32.25,3.7500,True,0,False,False,1525
1,malawi_2016,-17.117607882102988_35.17229723579403_-17.0951...,-17.117608,35.172297,-17.095150,35.217213,0.0,32.25,3.7500,True,0,False,False,2967
2,malawi_2016,-17.09515_35.17229723579403_-17.09515_35.21721...,-17.095150,35.172297,-17.095150,35.217213,0.0,32.25,3.7500,True,0,False,False,141
3,malawi_2016,-17.072692117897013_35.17229723579403_-17.0951...,-17.072692,35.172297,-17.095150,35.217213,0.0,32.25,3.7500,True,0,False,False,10129
4,malawi_2016,-17.050234235794026_35.17229723579403_-17.0951...,-17.050234,35.172297,-17.095150,35.217213,0.0,32.25,3.7500,True,0,False,False,3205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13625,malawi_2016,-9.474582764205977_33.06703376420597_-9.429667...,-9.474583,33.067034,-9.429667,33.022118,0.0,40.25,4.3125,True,1,False,True,2731
13626,malawi_2016,-9.45212488210299_33.06703376420597_-9.429667_...,-9.452125,33.067034,-9.429667,33.022118,0.0,40.25,4.3125,True,1,False,True,6893
13627,malawi_2016,-9.429667_33.06703376420597_-9.429667_33.02211...,-9.429667,33.067034,-9.429667,33.022118,0.0,40.25,4.3125,True,1,False,True,7030
13628,malawi_2016,-9.407209117897011_33.06703376420597_-9.429667...,-9.407209,33.067034,-9.429667,33.022118,0.0,40.25,4.3125,True,1,False,True,794


In [17]:
df_valid

,country,image_name,image_lat,image_lon,cluster_lat,cluster_lon,rCSI,FCS,HDDS,is_train,bin,near_lower,near_upper,feat_index
0,malawi_2016,-17.137266764205975_35.069727235794026_-17.092...,-17.137267,35.069727,-17.092351,35.114643,0.0625,32.18750,3.5625,False,0,False,False,5240
1,malawi_2016,-17.114808882102988_35.069727235794026_-17.092...,-17.114809,35.069727,-17.092351,35.114643,0.0625,32.18750,3.5625,False,0,False,False,2819
2,malawi_2016,-17.092351_35.069727235794026_-17.092351_35.11...,-17.092351,35.069727,-17.092351,35.114643,0.0625,32.18750,3.5625,False,0,False,False,3590
3,malawi_2016,-17.069893117897013_35.069727235794026_-17.092...,-17.069893,35.069727,-17.092351,35.114643,0.0625,32.18750,3.5625,False,0,False,False,871
4,malawi_2016,-17.047435235794026_35.069727235794026_-17.092...,-17.047435,35.069727,-17.092351,35.114643,0.0625,32.18750,3.5625,False,0,False,False,5766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5837,malawi_2016,-9.636293764205977_33.10236576420598_-9.591378...,-9.636294,33.102366,-9.591378,33.057450,1.1250,38.84375,4.1250,False,1,False,False,964
5838,malawi_2016,-9.61383588210299_33.10236576420598_-9.591378_...,-9.613836,33.102366,-9.591378,33.057450,1.1250,38.84375,4.1250,False,1,False,False,4285
5839,malawi_2016,-9.591378_33.10236576420598_-9.591378_33.05745...,-9.591378,33.102366,-9.591378,33.057450,1.1250,38.84375,4.1250,False,1,False,False,5052
5840,malawi_2016,-9.568920117897012_33.10236576420598_-9.591378...,-9.568920,33.102366,-9.591378,33.057450,1.1250,38.84375,4.1250,False,1,False,False,8


## Aggregate Features
For each country, we aggregate the image features per cluster and save them to results/country/cnn

In [18]:
def aggregate_features(df, feats, country, mode):
    assert mode in ['train', 'valid']
    group = df.groupby(['cluster_lat', 'cluster_lon'])
    x = np.zeros((len(group), 4096))
    cluster_list = [] # the corresponding clusters (lat, lon) to the x aggregate feature array
    for i, g in enumerate(group):
        lat, lon = g[0]
        im_sub = df[(df['cluster_lat'] == lat) & (df['cluster_lon'] == lon)].reset_index(drop=True)
        agg_feats = np.zeros((len(im_sub), 4096))
        for j, d in im_sub.iterrows():
            agg_feats[j,:] = feats[d.feat_index]
        agg_feats = agg_feats.mean(axis=0) # averages the features across all images in the cluster

        x[i,:] = agg_feats
        cluster_list.append([lat, lon])
        
    # save to the correct directory
    save_dir = os.path.join(RESULTS_DIR, 'cnn')
    os.makedirs(save_dir, exist_ok=True)
    print(f'saving to {save_dir}')
    np.save(os.path.join(save_dir, f'cluster_feats_{mode}_{METRIC}.npy'), x)
    pickle.dump(cluster_list, open(os.path.join(save_dir, f'cluster_order_{mode}_{METRIC}.pkl'), 'wb'))

In [19]:
aggregate_features(df_train, feats_train, COUNTRY, 'train')

saving to ../results/malawi_2016/HDDS/cnn


In [20]:
aggregate_features(df_valid, feats_valid, COUNTRY, 'valid')

saving to ../results/malawi_2016/HDDS/cnn
